In [1]:
! pip install stop_words

  Running setup.py bdist_wheel for stop-words ... done
  Stored in directory: /root/.cache/pip/wheels/22/74/80/77275c2f9f2f1d9841b51e169a38985640a10fbd2711d10791
Successfully built stop-words


In [2]:
# Import libraries
import numpy as np
import pandas as pd
import autosklearn.classification
import sklearn.model_selection
import sklearn.metrics
from stop_words import get_stop_words
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

In [3]:
# Load Dataset
df = pd.read_csv('../data/external/dataset.csv')

In [4]:
# Split data set into train and test
X = df[['name', 'description']]
y = df.category
X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)
X_train.shape
X_test.shape

(495, 2)

In [5]:
# Preprocess text data 
X_train.loc[:,"name"] = X_train.name.apply(lambda x : str.lower(x))
X_train.loc[:,"description"] = X_train.description.apply(lambda x : str.lower(x))
X_train.shape

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


(1485, 2)

In [6]:
# Encode train categorical variables as numeric values
labels = LabelEncoder()
y_train_label = labels.fit_transform(y_train)
y_train_label.shape

(1485,)

In [7]:
# Tokenize train text
count_vect = CountVectorizer(lowercase=False)
X_train_counts = count_vect.fit_transform(X_train.name, X_train.description)
X_train_counts.shape

(1485, 1760)

In [8]:
# From occurrences to frequencies
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1485, 1760)

In [ ]:
# Training classifier
clf = autosklearn.classification.AutoSklearnClassifier()
clf.fit(X_train_tfidf, y_train_label)

[WARNING] [2018-03-05 19:30:23,181:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-05 19:30:23,181:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-05 19:30:23,187:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-05 19:30:23,187:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


In [ ]:
# Create product dataframe
product = {"name": "Blusão Bichinho Chic Capuz Marrom","description": "Blusão Blusão Bichinho Chic Capuz Marrom Capuz, é uma Linda peça feito em soft brush, super macio e quentinho. Possui cordão para regulagem no pescoço e lindo bordado exclusiva Bichinho Chic."}
df_product = pd.DataFrame([dict(product)])

In [ ]:
# Tokenize product data
X_prod_counts = count_vect.transform(df_product)
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
description    1 non-null object
name           1 non-null object
dtypes: object(2)
memory usage: 96.0+ bytes


In [ ]:
# From occurrences to frequencies
X_prod_tfidf = tfidf_transformer.transform(X_prod_counts)
X_prod_tfidf.shape

(2, 1760)

In [ ]:
# Predict category from product name and description
y_hat = clf.predict(X_prod_tfidf)
y_hat

array([59, 59])

In [ ]:
# Encode test categorical variables as numeric values
y_test_label = labels.fit_transform(y_test)
y_test_label.shape

(495,)

In [ ]:
# Evaluation of the model's performance 
print("Accuracy score", accuracy_score(y_test_label, y_hat))

ValueError: Found input variables with inconsistent numbers of samples: [495, 2]

In [ ]:
# Model persistence
joblib.dump(clf, '../models/'+'classifier.pkl') 